In [1]:
import glob
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import rasterio as rio

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import imblearn as imb
from imblearn.under_sampling import RandomUnderSampler

import scipy
import itertools
import random
import ast

## Data prep

In [2]:
# Open img-scout dfs

data_dir = '/Users/kathleenkanaley/Desktop/grapes_from_space/data/'
data_20 = pd.read_csv(data_dir +'img_scout_dfs/2020/smr_vis_skysat_scout_2020.csv').dropna()
data_21 = pd.read_csv(data_dir +'img_scout_dfs/2021/smr_vis_skysat_scout_2021.csv').dropna()
data_22 = pd.read_csv(data_dir +'img_scout_dfs/2022/smr_vis_skysat_scout_2022.csv').dropna()

data_21 = data_21[data_21['Date']!='2021-08-03']
data_20 = data_20[data_20['Date']!='2020-09-09']

img_dfs = [data_20, data_21, data_22]
#all_years = pd.concat(img_dfs)

In [3]:
prepped_dfs = []
for df in img_dfs:
    df = df.rename(columns={"green_red": "grvi"})
    
    # Create binary numeric label
    # 0 = low GDM
    # 1 = high GDM

    df['GDM_sev']=0
    df.loc[df['DM_severity'] > 10, 'GDM_sev'] =1
    
    df['GDM_inc']=0
    df.loc[df['DM_inc'] > 25, 'GDM_inc'] = 1
    prepped_dfs.append(df)

In [4]:
# set up variables
data_2022 = prepped_dfs[2]
data_2021 = prepped_dfs[1]
data_2020 = prepped_dfs[0]
#data_all = pd.concat([data_2020,data_2021,data_2022])
#sev = 'severity_class'
inc = 'GDM_inc'
sbs = ['blue','green','red','nir']


In [5]:
def data_prep(data, expl_vars, resp_var, state, seed):
    X = data[expl_vars]
    y = data[resp_var]

    rus = RandomUnderSampler(random_state=state)
    X_rus, y_rus = rus.fit_resample(X, y)

    mm = MinMaxScaler()
    X_scaled = mm.fit_transform(X_rus)
    
    return X_scaled, y_rus #features (scaled) and labels (unscaled)

In [6]:
# def bal_split_norm(data, expl_vars, resp_var, state, seed):
#     X = data[expl_vars]
#     y = data[resp_var]

#     rus = RandomUnderSampler(random_state=state)
#     X_rus, y_rus = rus.fit_resample(X, y)

#     # Split dataset into training set and test set
#     X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus,
#                                                         test_size=0.3, 
#                                                         random_state=seed,
#                                                         stratify = y_rus)  # 70% training and 30% test
#     mm = MinMaxScaler()
#     X_train_scaled = mm.fit_transform(X_train)
#     X_test_scaled = mm.transform(X_test)
    
#     return X_train_scaled, y_train, X_test_scaled, y_test

In [7]:
# training data

Xtrain_2020,ytrain_2020 = data_prep(data_2020,sbs,inc,2020,14)


Xtrain_2021,ytrain_2021 = data_prep(data_2021,sbs,inc,2021,14)


Xtrain_2022,ytrain_2022 = data_prep(data_2022,sbs,inc,2022,14)

In [9]:
# testing data

mm=MinMaxScaler()

Xtest_2020all = mm.fit_transform(data_2020[sbs])
ytest_2020all = data_2020[inc].to_numpy().reshape(-1, 1)

Xtest_2021all = mm.fit_transform(data_2021[sbs])
ytest_2021all = data_2021[inc].to_numpy().reshape(-1, 1)

Xtest_2022all = mm.fit_transform(data_2022[sbs])
ytest_2022all = data_2022[inc].to_numpy().reshape(-1, 1)

In [10]:
# hyperparams

hp_2020 = {'bootstrap': True,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 34,
 'n_estimators': 200}

hp_2021 = {'bootstrap': True,
 'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 18,
 'n_estimators': 100}

hp_2022 = {'bootstrap': True,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

hp_all = {'bootstrap': True,
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 18,
 'n_estimators': 300}

In [11]:
# Create RF classifiers
rfc_2020 = RandomForestClassifier().set_params(**hp_2020)
rfc_2021 = RandomForestClassifier().set_params(**hp_2021)
rfc_2022 = RandomForestClassifier().set_params(**hp_2022)

In [12]:
# Fit RF classifiers
rfc_2020.fit(Xtrain_2020, ytrain_2020)
rfc_2021.fit(Xtrain_2021, ytrain_2021)
rfc_2022.fit(Xtrain_2022, ytrain_2022)

RandomForestClassifier(max_depth=3, n_estimators=200)

In [13]:
# 2020 applied to 2021
ypredict_20on21 = rfc_2020.predict(Xtest_2021all)
cf_matrix = confusion_matrix(ytest_2021all, ypredict_20on21)
print('Classification metrics\n train = 2020, test=2021\n')
print(cf_matrix)
report = classification_report(ytest_2021all, ypredict_20on21)
print(report)

# 2020 applied to 2022
ypredict_20on22 = rfc_2020.predict(Xtest_2022all)
cf_matrix = confusion_matrix(ytest_2022all, ypredict_20on22)
print('Classification metrics\n train = 2020, test=2022\n')
print(cf_matrix)
report = classification_report(ytest_2022all, ypredict_20on22)
print(report)

Classification metrics
 train = 2020, test=2021

[[179 612]
 [ 68  97]]
              precision    recall  f1-score   support

           0       0.72      0.23      0.34       791
           1       0.14      0.59      0.22       165

    accuracy                           0.29       956
   macro avg       0.43      0.41      0.28       956
weighted avg       0.62      0.29      0.32       956

Classification metrics
 train = 2020, test=2022

[[254 440]
 [  3  23]]
              precision    recall  f1-score   support

           0       0.99      0.37      0.53       694
           1       0.05      0.88      0.09        26

    accuracy                           0.38       720
   macro avg       0.52      0.63      0.31       720
weighted avg       0.95      0.38      0.52       720



In [14]:
## 2021 applied to 2020
ypredict_21on20 = rfc_2021.predict(Xtest_2020all)
cf_matrix = confusion_matrix(ytest_2020all, ypredict_21on20)
print('Classification metrics\n train = 2021, test=2020\n')
print(cf_matrix)
report = classification_report(ytest_2020all, ypredict_21on20)
print(report)

## 2021 applied to 2022
ypredict_21on22 = rfc_2021.predict(Xtest_2022all)
cf_matrix = confusion_matrix(ytest_2022all, ypredict_21on22)
print('Classification metrics\n train = 2021, test=2022\n')
print(cf_matrix)
report = classification_report(ytest_2022all, ypredict_21on22)
print(report)

Classification metrics
 train = 2021, test=2020

[[311  45]
 [ 55  63]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       356
           1       0.58      0.53      0.56       118

    accuracy                           0.79       474
   macro avg       0.72      0.70      0.71       474
weighted avg       0.78      0.79      0.79       474

Classification metrics
 train = 2021, test=2022

[[483 211]
 [ 14  12]]
              precision    recall  f1-score   support

           0       0.97      0.70      0.81       694
           1       0.05      0.46      0.10        26

    accuracy                           0.69       720
   macro avg       0.51      0.58      0.45       720
weighted avg       0.94      0.69      0.79       720



In [15]:
## 2022 applied to 2020
ypredict_22on20 = rfc_2022.predict(Xtest_2020all)
cf_matrix = confusion_matrix(ytest_2020all, ypredict_22on20)
print('Classification metrics\n train = 2022, test=2020\n')
print(cf_matrix)
report = classification_report(ytest_2020all, ypredict_22on20)
print(report)

## 2022 applied to 2021
ypredict_22on21 = rfc_2022.predict(Xtest_2021all)
cf_matrix = confusion_matrix(ytest_2021all, ypredict_22on21)
print('Classification metrics\n train = 2022, test=2021\n')
print(cf_matrix)
report = classification_report(ytest_2021all, ypredict_22on21)
print(report)

Classification metrics
 train = 2022, test=2020

[[135 221]
 [ 62  56]]
              precision    recall  f1-score   support

           0       0.69      0.38      0.49       356
           1       0.20      0.47      0.28       118

    accuracy                           0.40       474
   macro avg       0.44      0.43      0.39       474
weighted avg       0.57      0.40      0.44       474

Classification metrics
 train = 2022, test=2021

[[361 430]
 [ 35 130]]
              precision    recall  f1-score   support

           0       0.91      0.46      0.61       791
           1       0.23      0.79      0.36       165

    accuracy                           0.51       956
   macro avg       0.57      0.62      0.48       956
weighted avg       0.79      0.51      0.57       956

